In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from mpl_toolkits import mplot3d

**PSD, FFT**

In [ ]:
#Name of the file
name='best_777_T_Bz=5A_Bx=0.08A_exc810_det852.3_5mW_5K_sigma+_PEM_(30min).txt'

exp_data = np.loadtxt(name, skiprows=1, delimiter='\t')
t=exp_data[:,0]
rho=exp_data[:,3]

#FFT
sp = np.fft.fft(rho)
freq_fft = np.fft.fftfreq(t.shape[-1])
power_fft=(sp.real)**2
power_fft[0]=0

freq, power = signal.periodogram(rho, scaling='spectrum')
#freq, power = signal.welch(rho, scaling='spectrum')

font = {'size': 20}

#Main signal
plt.figure(figsize=(15, 5))
plt.title(r'Signal')
plt.ylabel("$p_c$(%)")
plt.xlabel("Time (s)")
plt.plot(t, rho, 'g', lw=2)
plt.xlim([0,max(t)])
plt.rc('font', **font)

#Power spectrum
plt.figure(figsize=(15, 5))
plt.title(r'Power spectral density')
plt.semilogy(freq, power, 'k')
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD [V**2/Hz]')
axes = plt.gca()
axes.set_xlim([0,0.5])
axes.set_ylim([10**-15,max(power)+max(power)*10])

#FFT
plt.figure(figsize=(15, 5))
plt.title(r'FFT')
plt.plot(freq, power, 'r')
axes = plt.gca()
axes.set_xlim([0,0.5])
axes.set_ylim([0,max(power)+max(power)*0.1])
axes.fill_between(freq, 0, power, facecolor='red')
plt.xlabel(r'Frequency (Hz)')
plt.ylabel(r'(arb. un.)')
plt.yticks([])

**Autocorrelation**

The autocorrelation of a signal describes the similarity of a signal against a time-shifted version of itself. For a signal $x$, the autocorrelation $r$ is:
$$r(k)=∑_nx(n)x(n−k)$$
 
In this equation, $k$ is often called the lag parameter.  $r(k)$ is maximized at  $k=0$ and is symmetric about $k$.
The autocorrelation is useful for finding repeated patterns in a signal. For example, at short lags, the autocorrelation can tell us something about the signal's fundamental frequency. For longer lags, the autocorrelation may tell us something about the tempo of a musical signal.

In [ ]:
N = len(rho)
fvi = np.fft.fft(rho, n=2*N)
acf = np.real(np.fft.ifft(fvi * np.conjugate(fvi))[:N])
acf = acf/(N - np.arange(N))

plt.figure(figsize=(15, 5))
plt.title(r'Autocorrelation')
plt.xlabel(r'Time delay, $t$')
plt.ylabel(r'$r(t)$')
plt.plot(t[0:len(acf)-int(len(acf)/2)], acf[0:len(acf)-int(len(acf)/2)], 'r')
plt.grid(color='black', linewidth=1)
plt.rc('font', **font)


**Embedding time**

The time delayed mutual information was suggested by Fraser and Swinney as a tool to determine a reasonable delay: Unlike the autocorrelation function, the mutual information takes into account also nonlinear correlations. One has to compute
$$
S = - \sum_{ij} p_{ij}(\tau) ln \frac{p_{ij}(\tau)}{p_ip_j}, 
$$
where for some partition on the real numbers $p_{ij}$ is the probability to find a time series value in the $i$-th interval, and $p_{ij}(\tau)$ is the joint probability that an observation falls into the $i$-th interval and the observation time $\tau$ later falls into the $j$-th. In theory this expression has no systematic dependence on the size of the partition elements and can be quite easily computed. There exist good arguments that if the time delayed mutual information exhibits a marked minimum at a certain value of $\tau$, then this is a good candidate for a reasonable time delay. However, these arguments have to be modified when the embedding dimension exceeds two. Moreover, as will become transparent in the following sections, not all applications work optimally with the same delay. Since we are not really interested in absolute values of the mutual information here but rather in the first minimum, the minimal implementation given here seems to be sufficient.

In [ ]:
def mut(x, y, bins=32):
    """
    Calculates mutual information, I = S(x) + S(y) - S(x,y), between x and y.
    S(x) is the Shannon entropy.
    _____
    Input (x : array, y : array, bins : int {bins for histogram})
    Output (i : float {mutual information})
    """
    p_x = np.histogram(x, bins)[0]
    p_y = np.histogram(y, bins)[0]
    p_xy = np.histogram2d(x, y, bins)[0].flatten()

    #Recalculate frequencies into  the probabilities
    p_x = p_x[p_x > 0] / np.sum(p_x)
    p_y = p_y[p_y > 0] / np.sum(p_y)
    p_xy = p_xy[p_xy > 0] / np.sum(p_xy)

    #Calculate the Shannon entropies
    Sh_x = np.sum(p_x * np.log2(p_x))
    Sh_y = np.sum(p_y * np.log2(p_y))
    Sh_xy = np.sum(p_xy * np.log2(p_xy))

    return Sh_xy - Sh_x - Sh_y


def tdmut(x, tau, bins=32):
    """
    Calculate the time-delayed mutual information.
    _____
    Input (x : array {1d time series}, tau : int {maximal time delay}, bins : int {bins for histogram})
    Output (arr : float {array with the time-delayed mutual information})
    """
    n = len(x)
    
    tau_max = min(n, tau)

    arr = np.empty(tau_max)
    
    arr[0] = mut(x, x, bins)

    for i in range(1, tau_max):
        arr[i] = mut(x[:-i], x[i:], bins)

    return arr

tau = 20

mutual_inf = tdmut(rho, tau=tau, bins=2)

#Plotting the delayed mutual information
plt.figure(figsize=(15, 5))
plt.plot(mutual_inf, label=(r'Bins = 2'), c='r')
plt.xticks(np.arange(0, tau, 2.0))
plt.grid(color='black', linewidth=1)
plt.title(r'Delayed mutual information')
plt.xlabel(r'Delay, $\tau$')
plt.ylabel(r'$I(\tau)$')
plt.show()

**False Nearest Neighbors**

As a test for determinism - method of false nearest neighbors introduced by Kennel et al. (1992). It was originally developed as an alternative to the approach of observing the saturation in invariants to determine the minimum embedding dimension required to reconstruct an attractor. If the attractor has not unfolded properly, it will contain a large number of “false nearest neighbors,” i.e., points that are close together solely due to trajectory crossings caused by the projection of the attractor onto a phase space of smaller dimension. Such neighbors no longer remain close once we increase the embedding dimension, telling us that the attractor cannot be fully unfolded at the chosen embedding dimension. Operationally, we consider a point $y^{(d)}_i$ and its nearest neighbor $y^{(d)}_{n (i,d)}$ in the $d$-dimensional reconstructed phase space as false if
$$
\frac{|x_{i+d\tau} - x_{n(i,d)+d\tau}|} {||y^{(d)}_i - y^{(d)}_{n(i,d)} ||}>A,
$$
where $||\cdot||$ is the Euclidean norm and $A$ is a suitable threshold. We test this condition for all points in the phase space and compute the total fraction of false near neighbors for each $d$. If this fraction becomes zero at a finite, and preferably small, $d$ (the minimum embedding dimension $d_{min}$), then we can conclude that the attractor has properly unfolded and that the time series comes out of a deterministic process. In other words, the system generating the time series is low-dimensional enough to allow its attractor to be embedded in a phase space of $d_{min}$ dimensions.

In [ ]:
from scipy.spatial import cKDTree as KDTree

#Delay from mutual information
r_delay=4

#Number of calculated embedding dimension:
m0=10
dim = np.arange(1, m0 + 1)


def delayed_vectros(x, dim, tau):
    """
    Calculates the time-delayed vectors from a time series.
    _____
    Input (x : array {time series}, dim : int {embedding dimension}, tau : int {time delay})
    Output (n-dim array {time-delayed array})
    """
    m = len(x) - tau * (dim - 1)
        
    return np.asarray([x[i:i + tau * (dim - 1) + 1:tau] for i in range(m)])


def n_neighbors(x, window=0):
    """
    Finds the nearest neighbors for all points in the array with KDTree search (SciPy).
    _____
    Input (y : n-dim array {time-delayed vectors}, window : int {Theiler window})
    Output (indices : array {indices of near neighbors}, distances : array {neighbor distances})
    """
    n = len(x)
    max_n = 2 * (window + 1) + 1
    indices = np.empty(n, dtype=int)
    distances = np.empty(n)
    
    tree = KDTree(x)
    
    for i, j in enumerate(x):
        
        for k in range(2, max_n + 2):
            dist, index = tree.query(j, k=k, p=np.inf)
            valid = (np.abs(index - i) > window) & (dist > 0)

            if np.count_nonzero(valid):
                distances[i] = dist[valid][0]
                indices[i] = index[valid][0]
                break
                
    return np.squeeze(indices), np.squeeze(distances)


def fnn(d, x, tau, threshold=10):
    y1 = delayed_vectros(x[:-tau], d, tau)
    y2 = delayed_vectros(x, d + 1, tau)

    index, dist = n_neighbors(y1)

    # Kennel's test
    f = np.abs(y2[:, -1] - y2[index, -1]) / dist > threshold

    return np.mean(f)
    

f_nn = [100 * fnn(i, rho, tau=r_delay) for i in dim]

#Plotting the FNN
plt.figure(figsize=(15, 5))
plt.title(r'False nearest neighbors')
plt.xlabel(r'Embedding dimension, $d$')
plt.ylabel(r'FNN (%)')
plt.plot(dim, f_nn, 'ro-')
plt.grid(color='black', linewidth=1)
plt.show()

**Phase space reconstruction**

In [ ]:
from ipywidgets import interact, FloatSlider

# Minimum FNN
m_fnn=6

# Filter
#sos = signal.butter(2, 0.5, 'lp', output='sos')
#rho1 = signal.sosfilt(sos, rho)
b, a = signal.butter(5, 0.25)
rho1 = signal.filtfilt(b, a, rho)

X = rho1[:-r_delay*2]
Y = rho1[r_delay:-r_delay]
Z = rho1[2*r_delay:]

plt.rcParams["figure.figsize"] = [10, 10]

step=1

def turn(xx,yy):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    #ax.set_xlabel(r'$x(t)$')
    #ax.set_ylabel(r'$x(t + \tau)$')
    #ax.set_zlabel(r'$x(t + 2\tau)$');

    ax.view_init(xx, yy)
    ax.plot(X[::step], Y[::step], Z[::step],'-r', lw=0.5)
    ax.scatter(X, Y, Z, c=np.sqrt(X**2+Y**2), cmap='magma', s=10)
    ax.set_facecolor('k')
    plt.axis('off')
    #plt.savefig(name+'3D1'+'.pdf')
    
interact(turn,xx=FloatSlider(value=55, min=0, max=360, step=1),
         yy=FloatSlider(value=75, min=0, max=90, step=1))

**Correlation sum and correlation dimension**

There are several ways to to quantify the self-similarity of a geometrical object by a dimension, i.e. correlated dimension.
The correlation sum for a collection of pints $x_n$ in some vector space to be the fraction of all possible pairs of points which are closer than a given distance $\epsilon$ in a particular norm. The basic formula is 
$$
C(\epsilon) = \frac{2}{N(N-1)} \sum_{i=1}^N \sum_{j=i+1}^N \Theta (\epsilon -||x_i - x_j||),
$$
where $\Theta$ is Heaviside step function. The sum just counts the pairs $(x_i,x_j)$ whose distance is smaller than $\epsilon$. In the limit of an infinite amount of data ($N \rightarrow \infty$) and for smaller $\epsilon$, $C$ scales like a power law, $C(\epsilon) \sim \epsilon^D$, and the correlated dimension defined as
$$
d(N,\epsilon) = \frac{\partial \mathrm{ln} C (\epsilon,N)}{\partial  \mathrm{ln} \epsilon}, \\
        D = \lim\limits_{\epsilon \to 0} \lim\limits_{N \to \infty} d(N, \epsilon).
$$
Instead of directly inspecting the correlation sum plots, it is often customary to plot the local slope,
$$
        D_2^{(d)} = \frac{d log C_2^{(d)} (r)}{d log r},
$$
as a function of $r$ in order to identify this scaling region. Scaling at extreme length scales is often impaired by “edge effects,” i.e., at very small length scales, noise and poor statistics create fluctuations in $D_2^{(d)}(r)$, and at very large length scales, finiteness of attractor size causes it to drop to zero. A scaling region with a fairly constant $D_2(d)$ is generally found at intermediate length scales sandwiched between these two extremes. If the time series comes out of a low-dimensional chaotic process, then one expects $D_2(d)$ to converge to the $D_2$ of the underlying attractor within a sufficiently small $d$ (i.e., at the minimum embedding dimension). On the other hand, for stochastic processes, the delay vectors fill up the phase space, causing $D_2(d)$ to diverge with $d$. For the special case of uncorrelated noise, $D_2(d)$ would be equal to $d$.

*Correlation sum*

In [ ]:
from scipy.spatial import distance

def c2(x, start=0.1, end=10, step=100, window=10):
    """
    Computation the correlation sum of the time series using Grassberger & Procaccia algorithm(1983).
    _____
    Input (x : n-dim array {points in the phase space}, start : int {starting point of the progression}, 
           stop : int {ending point of the progression}, step : int {number of points between start and end},
           window : int {Theiler window})
    Output (r : array {distances for which correlation sums is calculated}, c2 : array {correlation sum C(r)})
    """
    ratio = (end / start) ** (1.0 / (step - 1))
    r = start * ratio ** np.arange(step)
    r = np.asarray(r)
    r = np.sort(r[r > 0])
    
    bins = np.insert(r, 0, -1)
    c = np.zeros(len(r))
    n = len(x)
    
    for i in range(n - window - 1):
        dists = distance.cdist([x[i]], x[i + window + 1:], metric='chebyshev')[0]
        c += np.histogram(dists, bins=bins)[0]
        
    pairs = 0.5 * (n - window - 1) * (n - window)
    c = np.cumsum(c) / pairs
    
    return r[c > 0], c[c > 0]

vectors = [delayed_vectros(rho, d, r_delay) for d in dim]

#Plotting the correlation sum
plt.figure(figsize=(15, 5))
plt.title(r'Correlation sum')
plt.xlabel(r'Distance, $r$')
plt.ylabel(r'C(r)')
for v in vectors:
    x, y = c2(v)
    plt.semilogx(x, y, 'r')
plt.grid(color='black', linewidth=1)
plt.show()

*Correlation dimension*

In [ ]:
def d2(r, c2, window=10):
    """
    Computation the correlation dimension of the time series (C(r) ~ r^D2).
    _____
    Input (x : array {distances for which correlation sums is been calculated}, c2 : array {correlation sums}, 
           window : int {Theiler window})
    Output (d2 : array {average D2 at each distance})
    """
    hwin = 3
    N = len(r) - 2 * hwin
    d2 = np.empty(N)
    x, y = np.log(r), np.log(c2)

    for i in range(N):
        p, q = x[i:i + 2 * hwin + 1], y[i:i + 2 * hwin + 1]
        A = np.vstack([p, np.ones(2 * hwin + 1)]).T
        d2[i] = np.linalg.lstsq(A, q)[0][0]

    return d2

#Plotting the correlation dimension
plt.figure(figsize=(15, 5))
plt.title(r'Correlation dimension')
plt.xlabel(r'Distance, $r$')
plt.ylabel(r'$D_2$')

for v in vectors:
    x, y = c2(v)
    plt.semilogx(x[3:-3], d2(x, y), 'r')
    
axes = plt.gca()
axes.set_xlim([0.1, 10])
axes.set_ylim([0, 5])
plt.grid(color='black', linewidth=1)
plt.show()